---
date: 2024-02-05
layout: post
title: Tools for Detecting Undefined Behavior in My Bad C Example
---

Try in colab:
https://colab.research.google.com/github/philzook58/philzook58.github.io/blob/master/pynb/2024-02-05-undefined_behavior.ipynb

I posted a [little tutorial](https://www.philipzucker.com/cbmc_tut/) on [CBMC](https://github.com/diffblue/cbmc), the C Bounded model checker last week.

It made it to the front page of [Hacker news](https://news.ycombinator.com/item?id=39191507), which is neat. What I didn't expect is how people latched onto the uninitialized read in this example

In [39]:
%%file /tmp/ex1.c
#include <assert.h>
int main(){
    int x;
    if (x <= 42){
            assert(x != 12345);
    }
}


Writing /tmp/ex1.c


This is a serious issue that CBMC does not check for. It is a tool that can fairly easily and effectively answer certain kinds of questions, but not all. It is in the [process of getting better](https://github.com/diffblue/cbmc/issues/7732)

CBMC on it's own is insufficient to guarantee that a program does what you want and what you expect or that it obeys best practices. Defense in depth is important. We're awhiles off from something that reads your mind and makes your thoughts precise and cogent.

In [ ]:
%%bash
# download and install CBMC
wget https://github.com/diffblue/cbmc/releases/download/cbmc-5.95.1/ubuntu-20.04-cbmc-5.95.1-Linux.deb
apt-get install bash-completion
dpkg -i ubuntu-20.04-cbmc-5.95.1-Linux.deb

In [40]:
! cbmc /tmp/ex1.c

CBMC version 5.95.1 (cbmc-5.95.1) 64-bit x86_64 linux
Parsing /tmp/ex1.c
Converting
Type-checking ex1
Generating GOTO Program
Adding CPROVER library (x86_64)
Removal of function pointers and virtual functions
Generic Property Instrumentation
Running with 8 object bits, 56 offset bits (default)
Starting Bounded Model Checking
Runtime Symex: 0.00265417s
size of program expression: 23 steps
simple slicing removed 5 assignments
Generated 1 VCC(s), 1 remaining after simplification
Runtime Postprocess Equation: 1.2691e-05s
Passing problem to propositional reduction
converting SSA
Runtime Convert SSA: 0.0010266s
Running propositional reduction
Post-processing
Runtime Post-process: 4.841e-06s
Solving with MiniSAT 2.2.1 with simplifier
67 variables, 100 clauses
SAT checker inconsistent: instance is UNSATISFIABLE
Runtime Solver: 1.542e-05s
Runtime decision procedure: 0.0014064s

** Results:
/tmp/ex1.c function main
[main.assertion.1] line 5 assertion x != 12345: SUCCESS

** 0 of 1 failed (1 iter

It was commented that your compiler's `-Wall -Wextra -Wpedantic` can check this issue. It is good to enable and deal with all warnings somehow. Indeed, this program is obviously suspicious. These checks will not catch all problems and cannot satch some things CBMC can.

In [42]:
! gcc -Wall -Wextra -Wpedantic /tmp/ex1.c -o /tmp/ex1

/tmp/ex1.c: In function ‘main’:
/tmp/ex1.c:4:8: warning: ‘x’ is used uninitialized []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wuninitialized-Wuninitialized]8;;]
    4 |     if (x <= 42){
      |        ^


Another interesting tool to sort of spread spectrum apply static analyses to projects is [CodeChecker](https://codechecker.readthedocs.io/en/latest/). This applies some lighter weight larger scale analyses like [cppcheck](https://cppcheck.sourceforge.io/), infer, and the compiler built in analyzers. It also makes nice little html pages and has some kind of CI story.

These are good things to run. Coming back clean is no guarantee of software quality, but it is a good sign.

In [ ]:
! python3 pip install codechecker #install codechecker

In [46]:
! CodeChecker check --build "clang /tmp/ex1.c"

[INFO 2024-02-05 15:26] - Starting build...
[INFO 2024-02-05 15:26] - Using CodeChecker ld-logger.
[INFO 2024-02-05 15:26] - Build finished successfully.
[INFO 2024-02-05 15:26] - Previous analysis results in '/tmp/tmpc7xy9pwi' have been removed, overwriting with current result
[INFO 2024-02-05 15:27] - Enabled checkers:
clangsa: alpha.security.cert.pos.34c, core.BitwiseShift, core.CallAndMessage, core.DivideZero, core.NonNullParamChecker, core.NullDereference, core.StackAddressEscape, core.UndefinedBinaryOperatorResult, core.VLASize, core.uninitialized.ArraySubscript, core.uninitialized.Assign, core.uninitialized.Branch, core.uninitialized.CapturedBlockVariable, core.uninitialized.NewArraySize, core.uninitialized.UndefReturn, cplusplus.InnerPointer, cplusplus.Move, cplusplus.NewDelete, cplusplus.NewDeleteLeaks, cplusplus.PlacementNew, cplusplus.PureVirtualCall, cplusplus.StringChecker, deadcode.DeadStores, nullability.NullPassedToNonnull, nullability.NullReturnedFromNonnull, optin.cpl

I am still suspicious of the exact truth on whether the example given is actually undefined behavior. I mentioned a couple tools for checking on undefined behavior. Here I'll try the [undefined behavior sanitizer](https://clang.llvm.org/docs/UndefinedBehaviorSanitizer.html) which comes with your compiler and the [Cerberus](https://www.cl.cam.ac.uk/~pes20/cerberus/) interpreter. The UBSanitizer injects runtime checks for undefined behavior happening. I am unsure how complete these checks are or whther they can be optimized away. They probably try to make sure they are as reliable as is possible.

The UB sanitizer does not take issue with this example, nor does cerberus. Curious.

In [47]:
! clang -fsanitize=undefined /tmp/ex1.c -o /tmp/ex1 && /tmp/ex1

In [ ]:
%%bash
# install cerberus. This needs to get ocaml first. Might take a bit
apt install opam
opam init -y
eval $(opam env)
git clone https://github.com/rems-project/cerberus.git
cd cerberus
apt-get install libgmp-dev
opam install -y --deps-only ./cerberus-lib.opam ./cerberus.opam
make

In [48]:
! cerberus --exec /tmp/ex1.c

Just to check that it isn't compiling away.

In [1]:
! objdump /tmp/ex1 -d | grep -A 20 main.:

000000000002ea38 <main>:
   2ea38:	55                   	push   %rbp
   2ea39:	48 89 e5             	mov    %rsp,%rbp
   2ea3c:	48 83 ec 10          	sub    $0x10,%rsp
   2ea40:	c7 45 fc 00 00 00 00 	movl   $0x0,-0x4(%rbp)
   2ea47:	83 7d f8 2a          	cmpl   $0x2a,-0x8(%rbp)
   2ea4b:	0f 8f 4c 00 00 00    	jg     2ea9d <main+0x65>
   2ea51:	81 7d f8 39 30 00 00 	cmpl   $0x3039,-0x8(%rbp)
   2ea58:	0f 84 05 00 00 00    	je     2ea63 <main+0x2b>
   2ea5e:	e9 35 00 00 00       	jmp    2ea98 <main+0x60>
   2ea63:	48 8d 3d c1 92 00 00 	lea    0x92c1(%rip),%rdi        # 37d2b <_ZL10kVptrCheck+0xf25>
   2ea6a:	48 8d 35 c5 92 00 00 	lea    0x92c5(%rip),%rsi        # 37d36 <_ZL10kVptrCheck+0xf30>
   2ea71:	ba 05 00 00 00       	mov    $0x5,%edx
   2ea76:	48 8d 0d c4 92 00 00 	lea    0x92c4(%rip),%rcx        # 37d41 <_ZL10kVptrCheck+0xf3b>
   2ea7d:	e8 4e 66 fd ff       	call   50d0 <__assert_fail@plt>
   2ea82:	31 c0                	xor    %eax,%eax
   2ea84:	a8 01                	test   $0x

Asserts that aren't triggering may be an odd case, so let's do something more straightforward: printing uninitialized values.

In [51]:
%%file /tmp/ex2.c
#include <stdio.h>
int main(){
    int x;
    printf("Value of x: %d\n", x);
}

Overwriting /tmp/ex2.c


The UB sanitizer still does not take issue.
Cerberus thinks this is unspecified behavior (which prints differently from undefined behavior)


In [52]:
! clang -fsanitize=undefined /tmp/ex2.c -o /tmp/ex2 && /tmp/ex2

Value of x: 0


In [53]:
! cerberus --exec /tmp/ex2.c

Value of x: UNSPEC


This slightly different program is however, quite bad. An uninitialized pointer is dereferenced and both methods complain accordingly.

In [2]:
%%file /tmp/ex3.c
#include <stdio.h>
int main(){
    int* x;
    printf("Value of x: %d\n", *x);
}

Overwriting /tmp/ex3.c


In [3]:
! clang -fsanitize=undefined /tmp/ex3.c -o /tmp/ex3 && /tmp/ex3

/tmp/ex3.c:4:32: runtime error: load of null pointer of type 'int'
SUMMARY: UndefinedBehaviorSanitizer: undefined-behavior /tmp/ex3.c:4:32 
UndefinedBehaviorSanitizer:DEADLYSIGNAL
==33249==ERROR: UndefinedBehaviorSanitizer: SEGV on unknown address 0x000000000000 (pc 0x563890a65a7f bp 0x7ffde3c0d8b0 sp 0x7ffde3c0d890 T33249)
==33249==The signal is caused by a READ memory access.
==33249==Hint: address points to the zero page.
    #0 0x563890a65a7f in main (/tmp/ex3+0x2ea7f) (BuildId: c9aba5b3a42c58e96d9e5e92c81c7a22b892dcf2)
    #1 0x7f73eb0abd8f in __libc_start_call_main csu/../sysdeps/nptl/libc_start_call_main.h:58:16
    #2 0x7f73eb0abe3f in __libc_start_main csu/../csu/libc-start.c:392:3
    #3 0x563890a3c3e4 in _start (/tmp/ex3+0x53e4) (BuildId: c9aba5b3a42c58e96d9e5e92c81c7a22b892dcf2)

UndefinedBehaviorSanitizer can not provide additional info.
SUMMARY: UndefinedBehaviorSanitizer: SEGV (/tmp/ex3+0x2ea7f) (BuildId: c9aba5b3a42c58e96d9e5e92c81c7a22b892dcf2) in main
==33249==ABORTIN

In [57]:
! cerberus --exec /tmp/ex3.c

/tmp/ex3.c:4:32: error: undefined behaviour: the operand of the unary '*' operator has an invalid value
    printf("Value of x: %d\n", *x);
                               ^~ 
§6.5.3.2#4, sentence 4: 
4   The unary * operator denotes indirection. If the operand points to a function, the result is
    a function designator; if it points to an object, the result is an lvalue designating the
    object. If the operand has type ``pointer to type'', the result has type ``type''. If an
    invalid value has been assigned to the pointer, the behavior of the unary * operator is
    undefined.102)
    Forward references: storage-class specifiers (6.7.1), structure and union specifiers
    (6.7.2.1).


# Bits and Bobbles

A true Achilles heal of all modelling and verification tools is the connection of the model back to physical reality or the mushy concept of human intent. A model or verification or optimization is only as good as it's weakest link. 

I do not have (and suspect there cannot be) a final satisfying answer to this problem. 

I also do not consider it to be a a complete demonstration of the futility or uselessness of checking of verification attempts. 


I am not an expert in undefined behavior. Take anything you read here with a grain of salt. (Honestly, take anything you read anywhere on this topic with a grain of salt. People say of a lot of confident wrong stuff.) This represents my best guess at the topic, using the best tools i know of.

First off, you don't have to be _that_ scared. A mistake about undefined behavior is an unlikely cause of death. We are more likely to get cancer from some kind of slovenly corporate oversight about deodorant ingredients or basking in the wrong sunbeam.

We all die some day anyway. It is hopefully a whiles off and won't be that horrifying or painful.

On the other hand, I would nearly guarantee C and C++ are in the bioweapon, medical radiation, and nuclear stack. So it only takes one real nasty bug or even misconception to kill us all.

Turn on s lot of [warnings](https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html) (`-Wall -Wextra -Wpedantic` is a start) when you are working in an unsafe language and deal with them. In principle, it is quite possible for the compiler to detect situations that humans are likely to be confused. It isn't perfect, but even precise language lawyering isn't perfect if it doesn't comply with this particular user's intuitions and intent.

<https://en.wikipedia.org/wiki/Undefined_behavior>

Here's my basic understanding.

Undefined behavior:
An imperative program is kind of like a `state -> state` function. If you were to write an interpreter of your target imperative language in a purely functional style, this comes very naturally. The interpreter has a type `program -> state -> state` and if you partially apply it, you get it's state transformer semantics. `x := x + 1;` gets parsed as something like `Assign("x", Plus(Var("x"), Lit(1))`.   It takes in an initial state, which is the value of variables and memory, and outputs a final state of new values of variables and memory.

unspecified behavior:
If the program is allowed to fail, then the final state is not defined. Then we can model programs as `state -> option state`.

implementation defined behavior:
A different thing programs can do is do something, but it can be kind of random. This is unspecified behavior. Programs can be different choices of order of operations. We can think of programs as `state -> list state` with a list of possible final states.
A slightly different notion is that programs can be implementation-defined. They still do something that isn't quite clear, but the implementation picks a consistent thing to do. We can model this purely functionally. 
`() -> option state`

It is not always obvious in unsafe languages where the boundary between totally well-defined, undefined, unspecified, and implementation defined.

I am also suspicious that many of the HN comments are not right. People will talk at great length about undefined behavior. It was claimed that this example is an instance of "undefined behavior". I believe that this may be (or perhaps should be) an instance of "unspecified behavior', a distinct and slightly less insidious cousin. If so, CBMC is treating it's emulation of unspecified behavior in fairly reasonably manner I believe.

See for example 2.6.1 https://robbertkrebbers.nl/research/thesis.pdf  Defect Report #260  https://www.open-std.org/jtc1/sc22/wg14/www/docs/dr_260.htm Defect Report is numbered #451 https://www.open-std.org/jtc1/sc22/wg14/www/docs/dr_451.htm

I may be just confusing the issue because I hate being shown to be wrong and ignorant on the internet by strangers. Could be. Could be.


John Regehr has many great blog posts around this an other topics
https://blog.regehr.org/archives/1234

This is not the whole story, maybe not even a _majority_ of the story. What is the difference between a "do nothing" program and one that prints `"Hello World`? In our current conception of state, it is not clear that printing is evident in final state. There are a couple of ways out of this. We could attempt to say that no no no, state is the entire _universe_ including our eyeballs and the screen. This is slightly actionable. but also a bit silly.
We could also abandon our entire conception of imperative programs as mathematical functions. This is a weird way to think about it for most of the population, and smacks of sophistry and great faith and love of traditional mathematics. Traditional mathematics _is_  the prime example of being complex and yet (varying degrees of) precise.
The other thing we can do is say that there is

The C and C++ standards. I have not mastered the art of reading these (yea, yea go ahead and rip into me, you jackals.). C11 https://port70.net/~nsz/c/c11/n1570.html#J

Also take a look at the [Compcert](https://compcert.org/) [Formal verification of a realistic compiler](https://xavierleroy.org/publi/compcert-CACM.pdf) [A formally verified compiler back-end](https://xavierleroy.org/publi/compcert-backend.pdf), 

C semantics
- The C standard formalized in Coq https://robbertkrebbers.nl/research/thesis.pdf
- K framework semantics https://github.com/kframework/c-semantics https://www.ideals.illinois.edu/items/34571
- https://www.cl.cam.ac.uk/~pes20/cerberus/ cerberus

 https://www.cis.upenn.edu/~stevez/papers/XZHH+20.pdf interaction trees. Algerbaic effect style modeling

 memory model semantics.


In [ ]:
%%bash
# various installations
#pip install codechecker
#apt install cppcheck
# face book infer
#VERSION=1.1.0; \
#curl -sSL "https://github.com/facebook/infer/releases/download/v$VERSION/infer-linux64-v$VERSION.tar.xz" \
#| sudo tar -C /opt -xJ && \
#sudo ln -s "/opt/infer-linux64-v$VERSION/bin/infer" /usr/local/bin/infer
# cerberus



https://news.ycombinator.com/item?id=39191507

-Wall -Wpedantic -Wextra -Werror

For undefined behavior detection, I have heard of these:
- UB sanitizer https://clang.llvm.org/docs/UndefinedBehaviorSanitizer.html this can be used with klee for more coverage, but that doesn't increase the UB

- Cerberus Semantics https://www.cl.cam.ac.uk/~pes20/cerberus/

- https://github.com/kframework/c-semantics

- https://github.com/TrustInSoft/tis-interpreter


https://wordsandbuttons.online/so_you_think_you_know_c.html

https://clang-analyzer.llvm.org/
https://codechecker.readthedocs.io/en/latest/
https://clang-analyzer.llvm.org/scan-build.html

https://stackoverflow.com/questions/7237963/is-there-a-c-implementation-that-detects-all-undefined-behavior

https://twitter.com/kees_cook/status/1588007082288353281?s=20&t=udFq9u7zLY-5-Ae6VrdqeQ
```
-Wall
-D_FORTIFY_SOURCE=2
-fsanitize=bounds fsanitize-undefined-trap-on-error
-fstrict-flex-arrays (GCC 13+, Clang 16+)
```

semgrep
codeql

frama-c

astree



## Clang-tidy
https://clang.llvm.org/extra/clang-tidy/

In [ ]:
! clang-tidy /tmp/ex1.c

In [32]:
%%file /tmp/ub.c
#include <assert.h>
#include <stdio.h>
int main(){
    int x;
    x++;
    printf("Value of x: %d\n", x);
    //int y = *(int *)NULL;
    //assert(x != 12345);
    //assert(1 == 0);
}

Overwriting /tmp/ub.c


In [36]:
! gcc -O3 -Wall -Wpedantic -Wextra  /tmp/ub.c -o /tmp/ub && /tmp/ub

/tmp/ub.c: In function ‘main’:
/tmp/ub.c:5:6: warning: ‘x’ is used uninitialized []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wuninitialized-Wuninitialized]8;;]
    5 |     x++;
      |     ~^~
Value of x: 1


In [37]:
! objdump -d /tmp/ub | grep -A 20 main.:

0000000000001060 <main>:
    1060:	f3 0f 1e fa          	endbr64 
    1064:	48 83 ec 08          	sub    $0x8,%rsp
    1068:	ba 01 00 00 00       	mov    $0x1,%edx
    106d:	bf 01 00 00 00       	mov    $0x1,%edi
    1072:	31 c0                	xor    %eax,%eax
    1074:	48 8d 35 89 0f 00 00 	lea    0xf89(%rip),%rsi        # 2004 <_IO_stdin_used+0x4>
    107b:	e8 d0 ff ff ff       	call   1050 <__printf_chk@plt>
    1080:	31 c0                	xor    %eax,%eax
    1082:	48 83 c4 08          	add    $0x8,%rsp
    1086:	c3                   	ret    
    1087:	66 0f 1f 84 00 00 00 	nopw   0x0(%rax,%rax,1)
    108e:	00 00 

0000000000001090 <_start>:
    1090:	f3 0f 1e fa          	endbr64 
    1094:	31 ed                	xor    %ebp,%ebp
    1096:	49 89 d1             	mov    %rdx,%r9
    1099:	5e                   	pop    %rsi
    109a:	48 89 e2             	mov    %rsp,%rdx
    109d:	48 83 e4 f0          	and    $0xfffffffffffffff0,%rsp


In [20]:
%%bash
cerberus --exec /tmp/ub.c
#cerberus --batch 

/tmp/ub.c:7:13: error: undefined behaviour: the operand of the unary '*' operator has an invalid value
    int y = *(int *)NULL;
            ^~~~~~~~~~~~~~~~~~~~~~ 
§6.5.3.2#4, sentence 4: 
4   The unary * operator denotes indirection. If the operand points to a function, the result is
    a function designator; if it points to an object, the result is an lvalue designating the
    object. If the operand has type ``pointer to type'', the result has type ``type''. If an
    invalid value has been assigned to the pointer, the behavior of the unary * operator is
    undefined.102)
    Forward references: storage-class specifiers (6.7.1), structure and union specifiers
    (6.7.2.1).


## Codechecker
https://github.com/Ericsson/codechecker
That compile_commands.json might be kind of interesting



In [15]:
%%bash
# instrument a build first
CodeChecker log -o /tmp/compile_commands.json --build "clang -o /tmp/ub /tmp/ub.c"
# Then run the analyzer
CodeChecker analyze /tmp/compile_commands.json -o /tmp/reports
CodeChecker parse /tmp/reports -e html -o /tmp/reports.html

# all in one
#CodeChecker check --build "make" --output ./reports --clean \
#    --enable sensitive

[INFO 2024-02-01 11:11] - Starting build...
[INFO 2024-02-01 11:11] - Using CodeChecker ld-logger.
[INFO 2024-02-01 11:11] - Build finished successfully.
[INFO 2024-02-01 11:11] - Enabled checkers:
clang-tidy: bugprone-assert-side-effect, bugprone-bool-pointer-implicit-conversion, bugprone-compare-pointer-to-member-virtual-function, bugprone-copy-constructor-init, bugprone-dangling-handle, bugprone-dynamic-static-initializers, bugprone-fold-init-type, bugprone-forward-declaration-namespace, bugprone-forwarding-reference-overload, bugprone-inaccurate-erase, bugprone-inc-dec-in-conditions, bugprone-incorrect-enable-if, bugprone-incorrect-roundings, bugprone-infinite-loop, bugprone-integer-division, bugprone-lambda-function-name, bugprone-macro-repeated-side-effects, bugprone-misplaced-operator-in-strlen-in-alloc, bugprone-misplaced-pointer-arithmetic-in-alloc, bugprone-misplaced-widening-cast, bugprone-move-forwarding-reference, bugprone-multiple-new-in-one-expression, bugprone-not-null-

CalledProcessError: Command 'b'# instrument a build first\nCodeChecker log -o /tmp/compile_commands.json --build "clang -o /tmp/ub /tmp/ub.c"\n# Then run the analyzer\nCodeChecker analyze /tmp/compile_commands.json -o /tmp/reports\nCodeChecker parse /tmp/reports -e html -o /tmp/reports.html\n\n# all in one\n#CodeChecker check --build "make" --output ./reports --clean \\\n#    --enable sensitive\n'' returned non-zero exit status 2.

## Cppcheck

https://cppcheck.sourceforge.io/
https://github.com/danmar/cppcheck

In [10]:
! cppcheck /tmp/ub.c

Checking /tmp/ub.c ...
/tmp/ub.c:4:5: error: Uninitialized variable: x [uninitvar]
    x++;
    ^


In [ ]:
%%file /tmp/blast.sh
CCOPTIONS = -Wall -Wpedantic -fanalzye -fsanitize=address,bounds,undefined
clang $CCOPTIONS $1 -o  /tmp.a.out && ./a.out 
gcc $CCOPTIONS $1 &&
#cppcheck --enable=all $1
valgrind --leak-check=full --show-leak-kinds=all ./a.out
# codechecker
# infer



https://github.com/verateam/vera Probably old
clang-tidy also runs analyzer? clang-tidy-diff.py


In [ ]:
%%bash
apt install opam
opam init -y
eval $(opam env)
git clone https://github.com/rems-project/cerberus.git
cd cerberus
opam install --deps-only ./cerberus-lib.opam ./cerberus.opam
make

In [3]:
%%bash
cd ~/Downloads/cerberus
opam install angstrom
make cerberus-bmc
# make all


[NOTE] Package angstrom is already installed (current version is 0.15.0).


[DUNE] cerberus-bmc


File "backend/bmc/bmc_utils.ml", lines 326-356, characters 2-3:
326 | ..(match pe_ with
327 |   | PEsym s -> PEsym s
328 |   | PEimpl impl1 -> PEimpl impl1
329 |   | PEval v -> PEval v
330 |   | PEconstrained cs ->
...
353 |   | PEis_unsigned pe -> PEis_unsigned (self( 1) pe)
354 |   | PEbmc_assume pe -> PEbmc_assume (self( 1) pe)
355 |   | PEare_compatible( pe1, pe2) -> PEare_compatible( (self( 1) pe1), (self( 2) pe2))
356 |   )..
Error (warning 8 [partial-match]): this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
(PEconv_int (_, _)|PEwrapI (_, _, _, _)|
PEcatch_exceptional_condition (_, _, _, _))
make: *** [Makefile:69: cerberus-bmc] Error 1


CalledProcessError: Command 'b'cd ~/Downloads/cerberus\nopam install angstrom\nmake cerberus-bmc\n'' returned non-zero exit status 2.

In [1]:
%%bash
cd ~/Downloads/cerberus
dune exec cerberus -- --help


CERBERUS(1)                     Cerberus Manual                    CERBERUS(1)



NAME
       cerberus - Cerberus C semantics

SYNOPSIS
       cerberus [OPTION]… [FILE]…

ARGUMENTS
       FILE
           source C or Core file

OPTIONS
       --agnostic
           Asks Cerberus to delay looking at implementation settings until as
           late as possible. This makes the pipeline somewhat implementation
           agnostic.

       --args="ARG1 ARG2 ..."
           List of arguments for the C program

       --ast=LANG1,...
           Pretty print the intermediate syntax tree for the listed languages
           (ranging over {cabs, ail, core}).

       --batch
           makes the execution driver produce batch friendly output

       -c  Run frontend generating a target '.co' core object file.

       --charon-batch
           makes the execution driver produce batch friendly output (for
           Charon)

       --concurrency
           Activate the C11 concurrency

       --cpp=CM

In [ ]:
%%file 
struct S {
  int i;
  char c;
} s;

int main(void) {
  return sizeof(*(&s));
}

In [ ]:
%%file
int main(void) {
  char a = 0;
  short int b = 0;
  return sizeof(b) == sizeof(a+b);
}

In [ ]:
int main(void) {
  char a = ' ' * 13;
  return a;
}
	

In [ ]:
int main(void) {
  int i = 16;
  return (((((i >= i) << i) >> i) <= i));
}

In [ ]:
int main(void) {
  int i = 0;
  return i++ + ++i;
}